# RTB Auction Analysis

This notebook analyzes programmatic auction dynamics and bid strategy performance.

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from simulation.auction import AuctionSimulator, MarketDynamics, generate_sample_requests
from strategies.fixed import FixedBidStrategy
from strategies.dynamic import DynamicBidStrategy
from strategies.ml_strategy import MLBidStrategy
from analytics.performance import PerformanceAnalyzer

# Set style
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
%matplotlib inline

## 1. Generate Auction Data

In [ ]:
# Generate 10,000 bid requests
bid_requests = generate_sample_requests(10000, seed=42)
print(f"Generated {len(bid_requests):,} bid requests")

# Analyze bid request distribution
devices = [req.device.device_type.value for req in bid_requests]
positions = [req.impression.position for req in bid_requests]

print(f"\nDevice Distribution:")
print(pd.Series(devices).value_counts())
print(f"\nPosition Distribution:")
print(pd.Series(positions).value_counts())

## 2. Run Strategy Simulations

In [ ]:
# Initialize simulator
market = MarketDynamics(avg_competitors=5, volatility=0.3)
simulator = AuctionSimulator(market_dynamics=market)

# Initialize strategies
strategies = {
    'Fixed ($2.50)': FixedBidStrategy(2.5),
    'Dynamic': DynamicBidStrategy(base_bid=1.5, max_bid=12.0, target_cpa=15.0),
    'ML-Powered': MLBidStrategy(base_bid=1.5, max_bid=12.0, target_cpa=15.0)
}

# Run simulations
results = {}
for name, strategy in strategies.items():
    print(f"Running {name}...")
    results[name] = simulator.simulate_batch(bid_requests, strategy)
    print(f"  Completed {len(results[name]):,} auctions")

## 3. Performance Comparison

In [ ]:
# Analyze performance
analyzer = PerformanceAnalyzer()
comparison = analyzer.compare_strategies(results)
print("\n" + "="*80)
print("STRATEGY PERFORMANCE COMPARISON")
print("="*80)
print(comparison.to_string(index=False))

## 4. Win Rate Analysis

In [ ]:
# Plot win rates
win_rates = {}
for name, res in results.items():
    wins = sum(1 for r in res if r.did_win)
    win_rates[name] = wins / len(res) * 100

plt.figure(figsize=(10, 6))
plt.bar(win_rates.keys(), win_rates.values(), color=['#8b5cf6', '#06b6d4', '#10b981'])
plt.title('Win Rate by Strategy', fontsize=16, fontweight='bold')
plt.ylabel('Win Rate (%)', fontsize=12)
plt.xlabel('Strategy', fontsize=12)
plt.ylim(0, 100)
for i, (name, rate) in enumerate(win_rates.items()):
    plt.text(i, rate + 2, f'{rate:.1f}%', ha='center', fontsize=11, fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 5. ROAS Comparison

In [ ]:
# Calculate ROAS for each strategy
roas_data = {}
for name, res in results.items():
    spend = sum(r.winning_price for r in res if r.did_win)
    revenue = sum(r.revenue for r in res)
    roas_data[name] = revenue / spend if spend > 0 else 0

plt.figure(figsize=(10, 6))
plt.bar(roas_data.keys(), roas_data.values(), color=['#8b5cf6', '#06b6d4', '#10b981'])
plt.title('Return on Ad Spend (ROAS) by Strategy', fontsize=16, fontweight='bold')
plt.ylabel('ROAS (x)', fontsize=12)
plt.xlabel('Strategy', fontsize=12)
for i, (name, roas) in enumerate(roas_data.items()):
    plt.text(i, roas + 0.1, f'{roas:.2f}x', ha='center', fontsize=11, fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Bid Distribution Analysis

In [ ]:
# Analyze bid distributions
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for idx, (name, res) in enumerate(results.items()):
    bids = [r.winner_bid for r in res if r.did_win and r.winner_bid]
    axes[idx].hist(bids, bins=30, color=['#8b5cf6', '#06b6d4', '#10b981'][idx], alpha=0.7, edgecolor='black')
    axes[idx].set_title(f'{name}', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('Bid Amount ($)', fontsize=11)
    axes[idx].set_ylabel('Frequency', fontsize=11)
    axes[idx].axvline(np.mean(bids), color='red', linestyle='--', linewidth=2, label=f'Mean: ${np.mean(bids):.2f}')
    axes[idx].legend()
    axes[idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Statistical Significance Testing

In [ ]:
# Test significance between strategies
strategy_names = list(results.keys())

print("\n" + "="*80)
print("STATISTICAL SIGNIFICANCE TESTS")
print("="*80 + "\n")

for i in range(len(strategy_names)):
    for j in range(i + 1, len(strategy_names)):
        sig_test = analyzer.statistical_significance(
            results[strategy_names[i]],
            results[strategy_names[j]],
            metric='win_rate'
        )
        
        print(f"{strategy_names[i]} vs {strategy_names[j]}:")
        print(f"  Win Rates: {sig_test['strategy_a_value']:.2%} vs {sig_test['strategy_b_value']:.2%}")
        print(f"  P-value: {sig_test['p_value']:.6f}")
        print(f"  Significant (p < 0.05): {sig_test['significant']}")
        print()

## 8. Key Insights

In [ ]:
# Generate insights for best performer
best_strategy = max(roas_data.items(), key=lambda x: x[1])[0]
metrics = analyzer.analyze_strategy(best_strategy, results[best_strategy], bid_requests)

print(f"\n🏆 BEST STRATEGY: {best_strategy}")
print("="*80)
insights = analyzer.generate_insights(metrics)
for insight in insights:
    print(f"  {insight}")

print(f"\n📊 DETAILED METRICS:")
print(f"  Win Rate: {metrics.win_rate:.2%}")
print(f"  Total Spend: ${metrics.total_spend:,.2f}")
print(f"  Total Revenue: ${metrics.total_revenue:,.2f}")
print(f"  Conversions: {metrics.conversions:,}")
print(f"  Avg CPA: ${metrics.avg_cpa:.2f}")
print(f"  ROAS: {metrics.roas:.2f}x")
print(f"  Budget Efficiency: {metrics.budget_efficiency:.2f}")